In [1]:
import investpy
import re
import pandas as pd
import numpy as np

### get sector, ticker, counter_id from malaysiastockbiz (msb)

In [2]:
def load_data():
    KL = "https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1="

    df = pd.concat(
        [pd.read_html(f"{KL}{i+1}")[8] for i in range(30)]
    )

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df["counter_id"] = df["Company"].map(lambda x: re.search("\((.*?)\)", x).group(1))
    df["company_slug"] = df["Company"].map(lambda x: re.search("^([^\s]+)", x).group(1))
    df["company_name"] = df["Company"].map(lambda x: re.search("\)(.+)$", x).group(1))

    df["is_ace"] = df["company_name"].map(lambda x: True if x[:3] == "ACE" else False)
    df["is_etf"] = df["company_name"].map(lambda x: True if x[:5] == "ETFMY" else False)
    df["is_leap"] = df["company_name"].map(lambda x: True if x[:4] == "LEAP" else False)
    df["is_main"] = df["company_name"].map(lambda x: True if x[:4] == "MAIN" else False)
    
    df["market"] = df["company_name"].map(lambda x: "MAIN" if x[:4] == "MAIN" else x)
    df["market"] = df["market"].map(lambda x: "ACE" if x[:3] == "ACE" else x)
    df["market"] = df["market"].map(lambda x: "ETF" if x[:3] == "ETF" else x)
    df["market"] = df["market"].map(lambda x: "ETFMY" if x[:5] == "ETFMY" else x)
    df["market"] = df["market"].map(lambda x: "LEAP" if x[:4] == "LEAP" else x)

    df["company_name"] = df["company_name"].map(lambda x: x[3:] if x[:3] == "ACE" else x)
    df["company_name"] = df["company_name"].map(lambda x: x[5:] if x[:5] == "ETFMY" else x)
    df["company_name"] = df["company_name"].map(lambda x: x[4:] if x[:4] == "LEAP" else x)
    df["company_name"] = df["company_name"].map(lambda x: x[4:] if x[:4] == "MAIN" else x)

# manual fix to align all as int
    df["counter_id"] = df["counter_id"].replace({"5235SS": "5235"})
    
# drop null columns    
    df.drop(columns = ["Company", "Shariah"], inplace=True, axis=0)

# better to get market cap,PE, DY from investpy    
#     df = df[["company_name", "company_slug", "counter_id", "Market Cap", "Last Price", "PE", 
#              "DY", "ROE", "is_main","is_ace", "is_leap", "is_equity",  "Sector"]]
    
    df = df[["company_name", "company_slug", "counter_id", "Sector", "market",
             "is_main","is_ace", "is_leap", "is_etf"]]
    
    df.drop_duplicates(inplace=True)
    return df

In [3]:
msb = load_data()
msb.reset_index(inplace=True)

### Read counter_id crawled

In [4]:
stocks = pd.read_json('counter_id.json')
stocks.columns = stocks.columns.str.lower()

In [5]:
#create counter_id_num to map sector and fill in leading zeros
stocks['counter_id_num'] = stocks['counter_id'].map(lambda x: re.search("\d{1,}", x).group(0))
stocks['counter_id_num'] = stocks['counter_id_num'].apply(lambda x: x.zfill(4))
stocks['counter_id'] = stocks['counter_id'].apply(lambda x: x.zfill(4))

### Merge

In [6]:
merge = pd.merge(stocks, msb, left_on="counter_id_num", right_on="counter_id", how="left")
merge.reset_index(inplace=True)
merge

,level_0,ticker,counter_id_x,counter_id_num,index,company_name,company_slug,counter_id_y,Sector,market,is_main,is_ace,is_leap,is_etf
0,0,MDCH,5090,5090,894.0,MEDIA CHINESE INTERNATIONAL LIMITED,MEDIAC,5090,Media,MAIN,True,False,False,False
1,1,AMMB,1015,1015,306.0,AMMB HOLDINGS BERHAD,AMBANK,1015,Banking,MAIN,True,False,False,False
2,2,CIMB,1023,1023,310.0,CIMB GROUP HOLDINGS BERHAD,CIMB,1023,Banking,MAIN,True,False,False,False
3,3,RHBC,1066,1066,332.0,RHB BANK BERHAD,RHBBANK,1066,Banking,MAIN,True,False,False,False
4,4,HLCB,1082,1082,316.0,HONG LEONG FINANCIAL GROUP BERHAD,HLFG,1082,Banking,MAIN,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,924,GREA,0208,0208,809.0,GREATECH TECHNOLOGY BERHAD,GREATEC,0208,Technology Equipment,ACE,False,True,False,False
925,925,MEST,0207,0207,515.0,MESTRON HOLDINGS BERHAD,MESTRON,0207,Metals,ACE,False,True,False,False
926,926,PARL,0022,0022,201.0,PARLO BERHAD,PARLO,0022,"Travel, Leisure & Hospitality",ACE,False,True,False,False
927,927,MTAG,0213,0213,522.0,MTAG GROUP BERHAD,MTAG,0213,Industrial Services,ACE,False,True,False,False


In [7]:
merge.drop(columns=['counter_id_num', 'counter_id_y', 'level_0', 'index'], axis=0, inplace=True) 
merge.rename(columns={'counter_id_x':'counter_id'}, inplace=True)

In [8]:
merge[merge['company_name'].isnull()]

,ticker,counter_id,company_name,company_slug,Sector,market,is_main,is_ace,is_leap,is_etf
94,PPTB,0047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,AMVE,5959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,CHST,5214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,KRBN,3115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,MAXW,5189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,MSPT,5150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,SELS,1783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,XING,5155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
558,YTLL,2577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719,SWAH,9865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
merge[merge['company_slug']=="OCR"]

,ticker,counter_id,company_name,company_slug,Sector,market,is_main,is_ace,is_leap,is_etf
530,OCRG,7071,OCR GROUP BERHAD,OCR,Construction,MAIN,True,False,False,False
760,OCRGq,7071PA,OCR GROUP BERHAD,OCR,Construction,MAIN,True,False,False,False


In [10]:
merge.to_pickle("stock_sector.pickle")

In [11]:
print(f"Main: {len(merge[merge.is_main==True])}")
print(f"Ace: {len(merge[merge.is_ace==True])}")
print(f"Leap: {len(merge[merge.is_leap==True])}")
print(f"ETF: {len(merge[merge.is_etf==True])}")
print(f"Total: {(len(merge[merge.company_slug.notnull()]))}")

# manually checked the list below in market, no longer listed
print(f"Delisted: {(len(merge[merge.company_slug.isnull()]))}")

Main: 791
Ace: 121
Leap: 6
ETF: 0
Total: 918
Delisted: 11


### number of stocks from bursa

https://www.bursamalaysia.com/listing/listing_resources/ipo/listing_statistic

In [ ]:
Year	Main Ace LEAP	Total
2020	771	130	32	933
2019	772	129	28	929
2018	783	119	13	915
2017	788	115	2	905

### other useful stuffs

In [12]:
investpy.get_stock_company_profile('cimb', country='malaysia')

{'url': 'https://www.investing.com/equities/bumiputra---commerce-holdings-bhd-company-profile',
 'desc': "CIMB Group Holdings Berhad is a Malaysia-based provider of banking products and financial services. The Company's major business activities are categorized into three segments: Consumer Banking, Wholesale Banking and Commercial Banking. Its Consumer Banking segment provides regular banking solutions to individual customers, and it covers both conventional and Islamic financial products and services, such as residential and non-residential property loans, motor vehicle financing, credit cards and wealth management. Its Commercial Banking segment offers various products and services to small and medium-scale enterprises (SMEs) that include core banking credit facilities, trade financing, remittance and foreign exchange, together with general deposit products. Its Wholesale Banking segment comprises Investment Banking, Corporate Banking, Treasury and Markets, Transaction Banking, as w

In [13]:
investpy.stocks.get_stock_information("cimb", country='malaysia')

,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date
0,CIMB,3.41,3.41 - 3.46,1.611000e+10,3.41,3.09 - 5.45,0.47,14379100.0,3.413000e+10,0.26 (7.62%),21596832.0,7.31,1.91,- 32.81%,9.922961e+09,20/05/2020


In [14]:
investpy.technical.technical_indicators(name='CIMB', country='malaysia', product_type='stock', interval='daily')

,technical_indicator,value,signal
0,RSI(14),42.5550,sell
1,"STOCH(9,6)",22.6700,sell
2,STOCHRSI(14),35.5460,sell
3,"MACD(12,26)",-0.1000,sell
4,ADX(14),24.4640,neutral
5,Williams %R,-82.9790,oversold
6,CCI(14),-61.3322,sell
7,ATR(14),0.0936,less_volatility
8,Highs/Lows(14),-0.0264,sell
9,Ultimate Oscillator,39.4220,sell


In [15]:
investpy.news.get_calendar(countries=['malaysia'])

""
